#### 개요
학습대상 레코드와 컬럼으로 구성된 데이터를 대상으로 전처리 수행

##### 진행사항
아래 내용은 하나의 클래스에서 수행됨
* STEP01: 공사비 데이터셋 전처리(NaN처리, 컬럼추가 등) 
* STEP02: 전주 데이터셋 전처리 및 공사비 데이터셋 병합

##### 향후 추가할 사항
* 전주/전선에 대한 순서 지정
* 사번 데이터 문자 정보 추가
* 좌표정보 처리
* 향후 작업이 필요할 경우 아래 코드 참조
  * aidd-back240313/batch/preprocessing.py

In [1]:
import re
import pandas as pd

import aidd.sys.config as cfg
from aidd.sys.utils import Logs, AiddException
from aidd.batch.data_manager import save_data
from aidd.batch.data_manager import load_pickle, save_pickle

# 이 부분은 IPYNB에서 작업할 경우 만 필요함
# * IPYNB에서 merge.ipynb와 preprocessing.ipynb가 끈어져 추가한 부분임
from aidd.batch.data_manager import get_merged_data

#### 공통 클래스 생성

In [2]:
class Preprocessing():
    def __init__(self, mdata, is_batch=True):
        self._logs = Logs('PROPROCESSING')
        self.mdata = mdata
        self.is_batch = is_batch
        self.pdf = None
        try:
            self._run()
        except AiddException as ae:
            raise AiddException('PREPROCESSING_ERR', super_err=ae)
        except Exception as e:
            raise AiddException('PREPROCESSING_SERR', super_err=e)
        finally:
            self._logs.stop()
        
    def _run(self):
        self._cons()
        self._pole()
        
    def _cons(self):
        logs = Logs('PP_CONS')
        df = self.mdata['CONS']
        logs.mid('SOURCE_SIZE', df.shape)
        
        # 결측치 처리
        df.fillna(0, inplace=True)
        
        # 일자정보 처리
        # * '최종변경일시'를 이용해 다양한 일자정보 컬럼 추가
        # * 참고로 일자정보가 날자형식이 아니면 날자형식으로 변환
        if df.LAST_MOD_DATE.dtype != '<M8[ns]':
            df.LAST_MOD_DATE = pd.to_datetime(df.LAST_MOD_DATE)
        df['YEAR'] = df.LAST_MOD_DATE.dt.year
        df['MONTH'] = df.LAST_MOD_DATE.dt.month
        df['DAY'] = df.LAST_MOD_DATE.dt.day
        df['DAYOFWEEK'] = df.LAST_MOD_DATE.dt.dayofweek
        df['DAYOFYEAR'] = df.LAST_MOD_DATE.dt.dayofyear
        df['YEAR_MONTH'] = df.LAST_MOD_DATE.dt.strftime("%Y%m").astype(int)
        
        # 사번정보 처리
        # '최종변경자사번(LAST_MOD_EID)'만 사용(최초등록자==최종변경자사번)
        df['EID_NUMBER'] = df.LAST_MOD_EID.apply(
            lambda x: re.findall(r'\d+', x)[0]
        ).astype(int)
        
        # 사업소명 숫자로 변환
        # 모델학습 만을 생각하면 rank()함수를 사용할 수 있지만, 
        # rank를 사용하면 서비스시에 같은 값 다른 ranking을 제공하기 때문에 문제 발생
        # 이를 해결하기 위해 학습시 사업소명을 저장하고 
        # 서비스시 해당 사업소명리스트를 이용해 숫자로 변경해줘야 함.
        if self.is_batch:
            olist = df.OFFICE_NAME.unique().tolist()
            save_pickle(olist, pmode='PP_MEM', fcode='OFFICE_LIST')
        else:
            olist = load_pickle(pmode='PP_MEM', fcode='OFFICE_LIST')
        o_idxs = []
        for oname in df.OFFICE_NAME:
            o_idxs.append(olist.index(oname))
        df['OFFICE_NUMBER'] = o_idxs
        
        # 필요한 컬럼 추출
        df = df[cfg.MODELING_COLS['CONS']['PP']]
        logs.mid('RESULT', df.shape)
        
        self.pdf = df
        # 전처리 데이터 저장
        if self.is_batch:
            save_data(df, pmode='PP', file_code='CONS')
        logs.stop()
        
    def _pole(self):
        logs = Logs('PP_POLE')
        df = self.mdata['POLE']
        logs.mid('SOURCE_SIZE', df.shape)
        
        # 코드형 데이터 One-Hot Encoding
        prefix = ['POLE_SHAPE', 'POLE_TYPE', 'POLE_SPEC']
        columns = [x+'_CD' for x in prefix]
        # 정수형 실수값(.0)을 정수로 변환
        # (OneHotEndcoding시 동일 컬럼명으로 만들기 위해)
        if df.POLE_SPEC_CD.dtype != 'float64':
            df.POLE_SPEC_CD = df.POLE_SPEC_CD.astype(float)
        
        df = pd.get_dummies(df, columns=columns, prefix=prefix)
        df = df.applymap(lambda x: int(x) if isinstance(x, bool) else x)
        df_cols = df.columns.tolist()
        if self.is_batch:
            # 서비스에서 사용하기 위해 학습 당시의 컬럼 정보를 저장
            save_pickle(df_cols, pmode='PP_MEM', fcode='POLE_ONE_HOT_COLS')
        else:
            # 학습 당시 컬럼정보를 불러와 없는 컬럼을 찾아서 0으로 채움
            modeling_cols = load_pickle(pmode='PP_MEM', fcode='POLE_ONE_HOT_COLS')
            append_cols = [x for x in modeling_cols if x not in df_cols]
            df.loc[:, append_cols] = 0
        logs.mid('PP_ONLY', df.shape)
            
        # 공사비별 전주 합산 데이터 계산
        ## 공사번호 리스트 생성
        unique_cons_ids = df.CONS_ID.unique()
        ## 공사번호별 전주 합산 데이터 저장
        cons_id_pole_sums = []
        ## 합산대상 컬럼 리스트
        sum_cols = [c for c in df.columns if c.startswith('POLE_')]
        ## 공사번호별 합산(시간이 좀 걸림, 14700건 데이터 기준 39초)
        for cid in unique_cons_ids:
            temp_df = df[df.CONS_ID == cid]
            pole_sums = temp_df[sum_cols].sum().values.tolist()
            cons_id_pole_sums.append([cid] + pole_sums)
        ## 공사번호별로 합산된 전주 정보를 데이터프레임으로 변환
        pole_sums_df = pd.DataFrame(
            cons_id_pole_sums,
            columns=['CONS_ID'] + sum_cols
        )
        
        # 공사비 데이터와 전주 그룹데이터 병합
        pdf = pd.merge(
            self.pdf, pole_sums_df,
            left_on='CONS_ID', right_on='CONS_ID', how='left'
        )
        logs.mid('RESULT', pdf.shape)
        
        self.pdf = pdf
        # 전처리 데이터 저장
        if self.is_batch:
            save_data(pdf, pmode='PP', file_code='POLE')
        logs.stop()


In [3]:
bdata = get_merged_data(pmode='MB')
odata = get_merged_data(pmode='MO')

[e15028c3e441][2024-03-15 17:29:27.542367] 모델 학습용 데이터 불러오기 시작
[e15028c3e441][2024-03-15 17:29:27.598627]   읽은 데이터: CONS, 데이터 크기: (14729, 10), 처리시간: 0:00:00.056102
[e15028c3e441][2024-03-15 17:29:27.649531]   읽은 데이터: POLE, 데이터 크기: (27896, 6), 처리시간: 0:00:00.050850
[e15028c3e441][2024-03-15 17:29:27.757757]   읽은 데이터: LINE, 데이터 크기: (30348, 11), 처리시간: 0:00:00.108170
[e15028c3e441][2024-03-15 17:29:27.777983]   읽은 데이터: SL, 데이터 크기: (17034, 6), 처리시간: 0:00:00.020125
[e15028c3e441][2024-03-15 17:29:27.778020] 모델 학습용 데이터 불러오기 종료, 최종 처리시간: 0:00:00.235673
[cb89ec9d0b86][2024-03-15 17:29:27.778163] 모델 학습용 데이터 불러오기 시작
[cb89ec9d0b86][2024-03-15 17:29:27.779857]   읽은 데이터: CONS, 데이터 크기: (3, 11), 처리시간: 0:00:00.001668
[cb89ec9d0b86][2024-03-15 17:29:27.781416]   읽은 데이터: POLE, 데이터 크기: (9, 6), 처리시간: 0:00:00.001525
[cb89ec9d0b86][2024-03-15 17:29:27.783013]   읽은 데이터: LINE, 데이터 크기: (9, 11), 처리시간: 0:00:00.001560
[cb89ec9d0b86][2024-03-15 17:29:27.784287]   읽은 데이터: SL, 데이터 크기: (3, 6), 처리시간: 0:00:00.001244
[cb89

In [4]:
bdata['POLE'].POLE_SPEC_CD.dtype

dtype('float64')

In [5]:
pb = Preprocessing(bdata)
po = Preprocessing(odata, is_batch=False)

[2cf656750d29][2024-03-15 17:29:27.810382] 데이터 전처리 시작
[71e28c390e56][2024-03-15 17:29:27.810485]   공사비 데이터 전처리 시작
[71e28c390e56][2024-03-15 17:29:27.810532]     공사비+설비갯수 원본 데이터 크기: (14729, 10)
[2cf656750d29][2024-03-15 17:29:27.926241] 데이터 전처리 종료, 최종 처리시간: 0:00:00.115855


AiddException:   배치 데이터 전처리 과정에서 시스템 에러가 발생했습니다.
  "['TOTAL_CONS_COST'] not in index"


In [ ]:
po.pdf

,CONS_ID,TOTAL_CONS_COST,LAST_MOD_DATE,LAST_MOD_EID,OFFICE_NAME,YEAR,MONTH,DAY,DAYOFWEEK,DAYOFYEAR,...,POLE_SHAPE_V,POLE_TYPE_1,POLE_TYPE_B,POLE_TYPE_E,POLE_TYPE_M,POLE_TYPE_R,POLE_SPEC_6.0,POLE_SPEC_8.0,POLE_SPEC_11.0,POLE_SPEC_16.0
0,477420204194,0,2020-12-23 11:50:39,MDE1900011,단양지사,2020,12,23,2,358,...,0,0,0,0,0,0,0,0,0,0
1,475920223725,0,2022-11-24 10:42:22,MDP2100032,음성지사,2022,11,24,3,328,...,0,0,0,0,0,0,0,0,0,0
2,474620226651,0,2023-04-04 08:27:05,MDP2300252,충주지사,2023,4,4,1,94,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
op_df = po.pdf
bp_df = pb.pdf[pb.pdf.CONS_ID.isin(op_df.CONS_ID)]

In [ ]:
op_df.CONS_ID

0    477420204194
1    475920223725
2    474620226651
Name: CONS_ID, dtype: int64

In [ ]:
sbp_df = bdata['CONS']
sbp_df = sbp_df[sbp_df.CONS_ID.isin(op_df.CONS_ID)]
sbp_df

,CONS_ID,TOTAL_CONS_COST,CONS_TYPE_CD,LAST_MOD_DATE,LAST_MOD_EID,OFFICE_NAME,CONT_CAP,ACC_TYPE_NAME,POLE_CNT,LINE_CNT,SL_CNT,YEAR,MONTH,DAY,DAYOFWEEK,DAYOFYEAR,YEAR_MONTH,EID_NUMBER,OFFICE_NUMBER
